# 1. Summarize the paper on BPE

# 2. In regular expressions, what does \d, \D, \w, \W, \s, \S, {n}, {n,m}, {n,}, {,m} mean?

\d - matches any decimal digit

\D - matches any non-digit character

\w - matches alphanumeric characters

\W - matches nonword character

\s - matches whitespace character

\S - matches any non-whitespace character

The brackets specify how many copy of the previous re should be matched. so {n} mean exactly n copies of the previous re should be matched. {n,m} means to match from n to m copies of the preceding re. Omitting m gives an infinite upper bound of repetitions, and omitting n gives a lower bound of 0 repititions.

# 3. Create a Python based tokenizer in NLTK that replaces contractions (I’m, You’re, didn’t) into the expanded forms (I am, You are, did not).

In [152]:
# I used ChatGPT for part of this function but it didn't work at first so I had to fix it.

import nltk
nltk.download('punkt')
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import word_tokenize


def expand_contractions(sentence):
    contractions = {
        "i'm": "i am",
        "you're": "you are",
        "he's": "he is",
        "she's": "she is",
        "it's": "it is",
        "we're": "we are",
        "they're": "they are",
        "that's": "that is",
        "there's": "there is",
        "don't": "do not",
        "doesn't": "does not",
        "didn't": "did not",
        "can't": "cannot",
        "couldn't": "could not",
        "won't": "will not",
        "wouldn't": "would not",
        "shouldn't": "should not",
        "mightn't": "might not",
        "might've": "might have",
        "mustn't": "must not",
        "you'd": "you would",
        "i'd": "i would"
    }
    tk = WhitespaceTokenizer()
    words = tk.tokenize(sentence)
    expanded = []
    for word in words:
        if word.lower() in contractions:
            expanded.append(contractions[word.lower()])
        else:
            expanded.append(word)
    expanded = " ".join(expanded)
    return word_tokenize(expanded)

# Example usage:
sentence = "I'm glad you're here. Didn't think you'd make it."
expanded = expand_contractions(sentence)
print(expanded)  # Output: "I am glad you are here. did not think you would make it."

['i', 'am', 'glad', 'you', 'are', 'here', '.', 'did', 'not', 'think', 'you', 'would', 'make', 'it', '.']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\17023\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 4. Implement the BPE algorithm with the following interface

Your normalization should expand the contractions you implemented in problem 3.

In [153]:
import re, collections
import nltk

USE_STOP_WORDS = False

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

In [171]:
class Tokenizer:
    
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
    
    def normalize(self, text):
        contractions = {
            "i'm": "i am",
            "you're": "you are",
            "he's": "he is",
            "she's": "she is",
            "it's": "it is",
            "we're": "we are",
            "they're": "they are",
            "that's": "that is",
            "there's": "there is",
            "don't": "do not",
            "doesn't": "does not",
            "didn't": "did not",
            "can't": "cannot",
            "couldn't": "could not",
            "won't": "will not",
            "wouldn't": "would not",
            "shouldn't": "should not",
            "mightn't": "might not",
            "might've": "might have",
            "mustn't": "must not",
            "you'd": "you would",
            "i'd": "i would"
        }
        #tk = WhitespaceTokenizer()
        #words = tk.tokenize(sentence)
        expanded = []
        for word in text:
            if word.lower() in contractions:
                expanded.append(contractions[word.lower()])
            else:
                expanded.append(word)
        expanded = " ".join(expanded)
        return word_tokenize(expanded)
    
    def train(self, corpus):
        tokens = [s.split() for s in corpus]
        if USE_STOP_WORDS:
            stopword_list=nltk.corpus.stopwords.words('english')
            tokens = [[w for w in s if w.lower() not in stopword_list] for s in tokens]

        vocab_set = set(''.join(corpus)).union(['</w>'])

        vocab = collections.defaultdict(int)
        for s in tokens:
            for w in s:
                vocab[' '.join(list(w)) + ' </w>'] += 1

        num_merges = self.vocab_size
        for i in range(num_merges):
#             if i % 100 == 0:
#                 print(f"Iteration = {i}")
            pairs = get_stats(vocab)
            best = max(pairs, key=pairs.get)
            vocab = merge_vocab(best, vocab)
            vocab_set = vocab_set.union([''.join(best)])
            
        word2int = {}
        for w in vocab_set:
            word2int[w] = len(word2int)
            
        return tokens, vocab_set, word2int
    
    def encode(self, word, vocab_set, tokens, word2int):
        subunits = list(word) + ['</w>']
        changed = True
        while changed:
            token = ''
            tokens = []
            for unit in subunits:
                if token + unit in vocab_set:
                    token = token + unit
                else:
                    tokens.append(token)
                    token = unit
            tokens.append(token)
            changed = len(subunits) != len(tokens)
            subunits = tokens
        return [word2int.get(token, 0) for token in tokens], tokens
    
    def decode(self, num, word2int):
        return list(word2int.keys())[list(word2int.values()).index(num)]

In [172]:
from nltk.corpus import brown

text = brown.words()

print(len(text), len(set(text)))

tokenizer = Tokenizer(5000)

1161192 56057


In [156]:
norm_text = tokenizer.normalize(text)
brown_tokens, v_set, w2int = tokenizer.train(norm_text)

In [157]:
encoding = tokenizer.encode('would', v_set, brown_tokens, w2int)
print(encoding)

([38], ['would</w>'])


In [174]:
num = 38
decoding = tokenizer.decode(num, w2int)
print(decoding)

would</w>


# 5. Using the package https://www.nltk.org/api/nltk.chat.html#module-nltk.chat, you will create a regular expression based chatbot that answers the following questions.  

    a. What’s the temperature today? (check for the temperature at weather channel)
    b. What’s my zip code? (based on the person’s location)
    c. How much is $19.99 in <currency>? (You can search Google for “exchange rate between us and <currency>”)
    d. What’s the definition of <word>? (you will need to look for the definition at Wikipedia or dictionary)
    
The chatbot structure from NLTK uses static string responses, you have to modify it (as in the example below) to allow for functional objects that can parse the web, for example.

Note that you will need to process HTML to create the answers, and you will use Beautiful Soup to do that. 

https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.html?highlight=select

In [158]:
from nltk.chat.util import Chat, reflections
import random
import requests
from bs4 import BeautifulSoup
import geocoder
from nltk.tokenize import word_tokenize
import re

def get_temp(prompt):
    location_url = "https://weather.com/weather/today/l/4fb2eb4b20edcb606887ba1528d1e7f0ca27f832876ed59016bbbf08547ad493"
    response = requests.get(location_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    temperature = soup.find('span', {'class': 'CurrentConditions--tempValue--MHmYY'}).get_text()
    return f"The current temperature in Santa Clara is {temperature}"

def get_zip_code(prompt):
    g = geocoder.ip('me')
    zipcode = g.postal

    return f"The current ZIP code is {zipcode}."

def google_search_url(query):
    query = query.replace(' ', '+')
    url = f"https://www.google.com/search?q={query}"
    return url

def currency_convert(prompt):
    prompt_url = google_search_url(prompt)
    response = requests.get(prompt_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    amount = soup.find('div', {'class': 'BNeawe iBp4i AP7Wnd'}).find('div', {'class': 'BNeawe iBp4i AP7Wnd'}).get_text()
    return f"The correct amount is {amount}"

def get_word(prompt):
    tokens = word_tokenize(prompt)
    index = -1
    while re.match(r"[\.\?\!);:]|mean(ing)?",tokens[index]):
        index -= 1
    return tokens[index]

def get_definition(prompt):
    word = get_word(prompt)
    api_key = "3ac25ed7-e378-4c27-aaa4-1128d7d12335"
    url = f"https://dictionaryapi.com/api/v3/references/collegiate/json/{word}?key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        definitions = response.json()[0].get("shortdef")
        print(f"The definition(s) of {word} is/are: ")
        for i, definition in enumerate(definitions):
            print(f"{i+1}. {definition}")
        return "I hope this answers your question."
    else:
        return "Sorry, the word could not be found in the dictionary."

responses = (
    (
        r"(hello(.*))|(good [a-zA-Z]+)|((.*)[Tt]emperature(.*))",
        (
            get_temp,
        ),
    ),
    (
        r"((.*)[zZ]ip [cC]ode(.*))|(.*)[zZ]ip(.*)|(.*)[lL]ocation(.*)|(.*)[wW]here(.*)",
        (
            get_zip_code,
        ),
    ),
    (
        r"(.*)\$?\d+(\.\d+)?%?(.*)|(.*)[dD]ollars?(.*)",
        (
            currency_convert,
        ),
    ),
    (
        r"(.*)defin(e|ition)(.*)|(.*)[Ww]ord(.*)|(.*)mean(ing)?(.*)",
        (
            get_definition,
        ),
    )
)

def respond(self, str):
    """
    Generate a response to the user input.
    :type str: str
    :param str: The string to be mapped
    :rtype: str
    """

    # check each pattern
    for (pattern, response) in self._pairs:
        match = pattern.match(str)

        # did the pattern match?
        if match:
            resp = response[0]
            resp = resp(str)
            resp = self._wildcards(resp, match)  # process wildcards

            # fix munged punctuation at the end
            if resp[-2:] == "?.":
                resp = resp[:-2] + "."
            if resp[-2:] == "??":
                resp = resp[:-2] + "?"
            return resp

chatbot = Chat(responses, reflections)
Chat.respond = respond

def chat():
    print("*" * 75)
    print("Chatbot!".center(75))
    print("*" * 75)
    print("Welcome.")

    chatbot.converse()
    
chat()

***************************************************************************
                                  Chatbot!                                 
***************************************************************************
Welcome.
>hello what is the temperature
The current temperature in Santa Clara is 53°
>Cool. Where am I?
The current ZIP code is 95138.
>how much money is 13 dollars in yuan
The correct amount is 89.84 Chinese Yuan
>Great. What is the definition of beautiful?
The definition(s) of beautiful is/are: 
1. having qualities of beauty : exciting aesthetic pleasure
2. generally pleasing : excellent
I hope this answers your question.
>quit
None


# 6. Implement the spell checker for cell phones

Have you ever tried to type in something quickly and because the keyboard size in the cell phone is much smaller than your finger, it typing in the neighboring letters?

   a) You will implement the spell checker from the site: https://norvig.com/spell-correct.html

   b) You will change your code to consider that replacements would only occur to neighboring words. For example, in the picture, the letter 'u' can be replaced by 'y' or 'i'.

![Keyboard iPhone](keyboard.png)

In [43]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [44]:
print(correction('katch'))
print(correction('boz'))
print(correction('ehat'))

watch
boy
that


In [51]:
import re
from collections import Counter

neighbors = {  # dictionary of neighboring words
    'a' : ('s', ['q', 'w', 'd']),
    'b' : (['v', 'n'], ['h', 'c', 'm']),
    'c' : (['x', 'v'], ['f', 'z', 'b']),
    'd' : (['s', 'f'], ['e', 'r', 'a', 'g', 'x']),
    'e' : (['w', 'r'], ['s', 'd', 'q', 't']),
    'f' : (['d', 'g'], ['r', 't', 'c', 's', 'h']),
    'g' : (['f', 'h'], ['t', 'y', 'v', 'd', 'j']),
    'h' : (['g', 'j'], ['y', 'u', 'b', 'f', 'k']),
    'i' : (['u', 'o'], ['j', 'k', 'y', 'p']),
    'j' : (['h', 'k'], ['u', 'i', 'n', 'g', 'l']),
    'k' : (['j', 'l'], ['i', 'o', 'm', 'h']),
    'l' : ('k', ['o', 'p', 'j']),
    'm' : ('n', ['k', 'b']),
    'n' : (['b', 'm'], ['j', 'v']),
    'o' : (['i', 'p'], ['k', 'l', 'u']),
    'p' : ('o', ['l', 'i']),
    'q' : ('w', ['a', 'e']),
    'r' : (['e', 't'], ['d', 'f', 'w', 'y']),
    's' : (['a', 'd'], ['w', 'e', 'z', 'f']),
    't' : (['r', 'y'], ['f', 'g', 'e', 'u']),
    'u' : (['y', 'i'], ['h', 'j', 't', 'o']),
    'v' : (['c', 'b'], ['g', 'x', 'n']),
    'w' : (['q', 'e'], ['a', 's', 'r']),
    'x' : (['z', 'c'], ['d', 'v']),
    'y' : (['t', 'u'], ['g', 'h', 'r', 'i']),
    'z' : ('x', ['s', 'c'])
}

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in neighbors[R[0]][0]]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [52]:
print(correction('katch'))
print(correction('boz'))
print(correction('ehat'))

latch
box
what


# 7. Implement the Weighted Mininum Edit Distance

In this problem, you will implement the weighted minimum edit distance algorithm of slide 94.

You will consider the following.

- delete and insertion cost is 1
- substitution cost is 1 if it is in adjacent in the keyboard, like in problem 6.b
- substitution cost is 2 if it is below or above, or two characters to the right or left (for example, in the example of 6.b, replacing a 'u' by a 't' or 'o' would have a cost of 2
- substitution cost is infinity if that does not apply

Run the algorithm for the two words: 

- 'caft' vs 'cat' 
- 'coffee' vs 'voffrt'

In [95]:
import numpy as np

def sub_cost(word1, word2, i, j):
    if word2[j-1] in neighbors[word1[i-1]][0]:
        return 1
    elif word2[j-1] in neighbors[word1[i-1]][1]:
        return 2
    elif word1[i-1] == word2[j-1]:
        return 0
    else:
        return 1000000000

def min_weight_dist(word1, word2):
    m, n = len(word1), len(word2)
    D = np.zeros((m+1,n+1))
    D[0,0] = 0
    
    for i in range(m+1):
        D[i,0] = i
    for j in range(n+1):
        D[0,j] = j
    
    for i in range(1,m+1):
        for j in range(1,n+1):
            delete = D[i-1,j] + 1
            insert = D[i,j-1] + 1
            substitute = D[i-1,j-1] + sub_cost(word1,word2,i,j)
            D[i,j] = min([delete, insert, substitute])
            #print(f"i = {i}, j = {j}, min = {min([delete, insert, substitute])}")
            #print(f"Matrix = \n{D}")
    
    return int(D[m,n])
            

In [96]:
print(min_weight_dist('caft','cat'))

1


In [98]:
print(min_weight_dist('coffee','voffrt'))

4


In [97]:
print(min_weight_dist('intention','execution'))

8
